In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, SpatialDropout1D
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [5]:
from google.colab import files
uploaded = files.upload()

Saving spam.csv to spam.csv


In [6]:
import pandas as pd
import io

df = pd.read_csv(io.BytesIO(uploaded['spam.csv']), encoding="latin-1")
df = df[['v1', 'v2']]
df.columns = ['label', 'message']
df['label'] = df['label'].map({'Not Spam': 0, 'Spam': 1})

In [7]:
df.head()

,label,message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [8]:
stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = re.sub(r'\W', ' ', text)
    text = text.lower()
    text = text.split()
    text = [word for word in text if word not in stop_words]
    return ' '.join(text)

df['message'] = df['message'].apply(clean_text)
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(df['message'])
sequences = tokenizer.texts_to_sequences(df['message'])
max_length = max(len(x) for x in sequences)
X = pad_sequences(sequences, maxlen=max_length, padding='post')
y = df['label'].values

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
model = Sequential()
model.add(Embedding(input_dim=5000, output_dim=64, input_length=max_length))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [15]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 19s 135ms/step - accuracy: 0.8667 - loss: 0.3928 - val_accuracy: 0.8655 - val_loss: 0.3952
Epoch 2/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 16s 102ms/step - accuracy: 0.8700 - loss: 0.3881 - val_accuracy: 0.8655 - val_loss: 0.3952
Epoch 3/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 20s 101ms/step - accuracy: 0.8644 - loss: 0.3974 - val_accuracy: 0.8655 - val_loss: 0.3959
Epoch 4/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 14s 100ms/step - accuracy: 0.8609 - loss: 0.4048 - val_accuracy: 0.8655 - val_loss: 0.3950
Epoch 5/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 14s 100ms/step - accuracy: 0.8664 - loss: 0.3939 - val_accuracy: 0.8655 - val_loss: 0.3968
Epoch 6/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 25s 130ms/step - accuracy: 0.8683 - loss: 0.3904 - val_accuracy: 0.8655 - val_loss: 0.3952
Epoch 7/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 17s 106ms/step - accuracy: 0.8666 - loss: 0.3926 - val_accuracy: 0.8655 - val_loss: 0.3952
Epoch 8/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 20s 101ms/step - accuracy: 0.8697 - loss: 0

In [30]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy * 100:.2f}%')

35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 50ms/step - accuracy: 0.1360 - loss: 0.6985
Accuracy: 15.07%


In [31]:
import re

def clean_text(text):
    text = re.sub(r'\W', ' ', text)
    text = text.lower().strip()
    return text

In [32]:
print(df['label'].value_counts())

label
0    4825
1     747
Name: count, dtype: int64


In [34]:
vocab_size = len(tokenizer.word_index) + 1
embedding_dim = 100

embedding_matrix = np.zeros((vocab_size, embedding_dim))

embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim,
                            weights=[embedding_matrix], input_length=max_length,
                            trainable=False)

In [35]:
import numpy as np
from tensorflow.keras.layers import Embedding

embedding_dim = 100
embedding_matrix = np.zeros((vocab_size, embedding_dim))

embedding_layer = Embedding(input_dim=vocab_size, output_dim=embedding_dim,
                            weights=[embedding_matrix], input_length=max_length,
                            trainable=False)

In [36]:
history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/15
140/140 ━━━━━━━━━━━━━━━━━━━━ 34s 180ms/step - accuracy: 0.8559 - loss: 0.3432 - val_accuracy: 0.9749 - val_loss: 0.0885
Epoch 2/15
140/140 ━━━━━━━━━━━━━━━━━━━━ 41s 183ms/step - accuracy: 0.9864 - loss: 0.0545 - val_accuracy: 0.9857 - val_loss: 0.0556
Epoch 3/15
140/140 ━━━━━━━━━━━━━━━━━━━━ 43s 194ms/step - accuracy: 0.9923 - loss: 0.0228 - val_accuracy: 0.9857 - val_loss: 0.0456
Epoch 4/15
140/140 ━━━━━━━━━━━━━━━━━━━━ 39s 183ms/step - accuracy: 0.9973 - loss: 0.0098 - val_accuracy: 0.9848 - val_loss: 0.0567
Epoch 5/15
140/140 ━━━━━━━━━━━━━━━━━━━━ 42s 193ms/step - accuracy: 0.9978 - loss: 0.0081 - val_accuracy: 0.9883 - val_loss: 0.0520
Epoch 6/15
140/140 ━━━━━━━━━━━━━━━━━━━━ 41s 193ms/step - accuracy: 0.9994 - loss: 0.0030 - val_accuracy: 0.9883 - val_loss: 0.0523
Epoch 7/15
140/140 ━━━━━━━━━━━━━━━━━━━━ 40s 184ms/step - accuracy: 0.9989 - loss: 0.0056 - val_accuracy: 0.9892 - val_loss: 0.0626
Epoch 8/15
140/140 ━━━━━━━━━━━━━━━━━━━━ 24s 173ms/step - accuracy: 0.9995 - loss: 0

In [38]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Accuracy: {accuracy * 100:.2f}%')

35/35 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step - accuracy: 0.9589 - loss: 0.1427
Accuracy: 95.25%


In [40]:
def predict_spam(text):
    text = clean_text(text)
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=max_length, padding='post')
    prediction = model.predict(padded)[0][0]
    return "Spam" if prediction > 0.3 else "Not Spam"

print(predict_spam("Congratulations! You won a lottery. Call now!"))
print(predict_spam("Hey, are we still meeting for lunch?"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Not Spam
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Not Spam


In [41]:
print(df['label'].value_counts())

label
0    4825
1     747
Name: count, dtype: int64


In [42]:
from sklearn.utils import resample

df_spam = df[df['label'] == 1]
df_ham = df[df['label'] == 0]

df_spam_upsampled = resample(df_spam, replace=True, n_samples=len(df_ham), random_state=42)

df_balanced = pd.concat([df_ham, df_spam_upsampled])

X_balanced = df_balanced['message']
y_balanced = df_balanced['label']

In [43]:
df_ham_downsampled = resample(df_ham, replace=False, n_samples=len(df_spam), random_state=42)

df_balanced = pd.concat([df_ham_downsampled, df_spam])

X_balanced = df_balanced['message']
y_balanced = df_balanced['label']

In [44]:
tokenizer.fit_on_texts(X_balanced)
X_sequences = tokenizer.texts_to_sequences(X_balanced)
X_padded = pad_sequences(X_sequences, maxlen=max_length, padding='post')

y_balanced = np.array(y_balanced)

In [45]:
history = model.fit(X_padded, y_balanced, epochs=15, batch_size=32, validation_split=0.2)

Epoch 1/15
38/38 ━━━━━━━━━━━━━━━━━━━━ 8s 215ms/step - accuracy: 0.8276 - loss: 0.4135 - val_accuracy: 0.8194 - val_loss: 0.4200
Epoch 2/15
38/38 ━━━━━━━━━━━━━━━━━━━━ 9s 177ms/step - accuracy: 0.9221 - loss: 0.2130 - val_accuracy: 0.8261 - val_loss: 0.3940
Epoch 3/15
38/38 ━━━━━━━━━━━━━━━━━━━━ 10s 165ms/step - accuracy: 0.9799 - loss: 0.0859 - val_accuracy: 0.9164 - val_loss: 0.2052
Epoch 4/15
38/38 ━━━━━━━━━━━━━━━━━━━━ 8s 215ms/step - accuracy: 0.9872 - loss: 0.0492 - val_accuracy: 0.9365 - val_loss: 0.1681
Epoch 5/15
38/38 ━━━━━━━━━━━━━━━━━━━━ 8s 164ms/step - accuracy: 0.9944 - loss: 0.0246 - val_accuracy: 0.9365 - val_loss: 0.2012
Epoch 6/15
38/38 ━━━━━━━━━━━━━━━━━━━━ 10s 162ms/step - accuracy: 0.9968 - loss: 0.0176 - val_accuracy: 0.9264 - val_loss: 0.2330
Epoch 7/15
38/38 ━━━━━━━━━━━━━━━━━━━━ 10s 166ms/step - accuracy: 0.9988 - loss: 0.0084 - val_accuracy: 0.9465 - val_loss: 0.1888
Epoch 8/15
38/38 ━━━━━━━━━━━━━━━━━━━━ 12s 219ms/step - accuracy: 0.9999 - loss: 0.0039 - val_accuracy

In [46]:
print(predict_spam("Congratulations! You won a lottery. Call now!"))
print(predict_spam("Hey, are we still meeting for lunch?"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Not Spam
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Not Spam


In [49]:
def predict_spam(text):
    text = clean_text(text)
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=max_length, padding='post')
    prediction = model.predict(padded)[0][0]
    return "Spam" if prediction > 0.5 else "Not Spam"

In [52]:
!pip install tensorflow-addons==0.23.0 typeguard==2.13.3 inflect==7.5.0 --no-deps

In [54]:
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test), batch_size=32)

Epoch 1/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 26s 183ms/step - accuracy: 0.9555 - loss: 0.1614 - val_accuracy: 0.9803 - val_loss: 0.0620
Epoch 2/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 40s 174ms/step - accuracy: 0.9945 - loss: 0.0258 - val_accuracy: 0.9785 - val_loss: 0.0589
Epoch 3/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 42s 183ms/step - accuracy: 0.9983 - loss: 0.0094 - val_accuracy: 0.9821 - val_loss: 0.0553
Epoch 4/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 42s 192ms/step - accuracy: 0.9978 - loss: 0.0112 - val_accuracy: 0.9848 - val_loss: 0.0632
Epoch 5/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 41s 190ms/step - accuracy: 0.9989 - loss: 0.0060 - val_accuracy: 0.9848 - val_loss: 0.0633
Epoch 6/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 25s 177ms/step - accuracy: 0.9988 - loss: 0.0085 - val_accuracy: 0.9830 - val_loss: 0.0625
Epoch 7/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 41s 180ms/step - accuracy: 0.9995 - loss: 0.0032 - val_accuracy: 0.9830 - val_loss: 0.0711
Epoch 8/10
140/140 ━━━━━━━━━━━━━━━━━━━━ 41s 183ms/step - accuracy: 0.9998 - loss: 0

In [59]:
def predict_spam(text):
    text = clean_text(text)
    seq = tokenizer.texts_to_sequences([text])
    padded = pad_sequences(seq, maxlen=max_length, padding='post')
    prediction = model.predict(padded)[0][0]
    return "Spam" if prediction > 0.3 else "Not Spam"

In [60]:
print(predict_spam("Congratulations! You won a lottery. Call now!"))
print(predict_spam("Hey, are we still meeting for lunch?"))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step
Spam
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step
Not Spam


In [61]:
model.save("spam_detection_model.h5")

In [62]:
from tensorflow.keras.models import load_model
model = load_model("spam_detection_model.h5")

In [65]:
from google.colab import files
files.download("spam_detection_model.h5")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>